In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [110]:
df = pd.read_csv(r'C:\Users\user\Desktop\Data\YouTube_data.csv')
df

,url,Timestamp,Title,Views,upload_date,Likes,Dislikes,Comments
0,https://www.youtube.com/watch?v=8g6GpkOvxZ8,27/07/2020 16:36:01,"GATE 2020 Topper | Hitesh Poply (CS, AIR-1) | ...","46,489 views",14 Mar 2020,1.7K,32,Coding Intelligence - I am pretty sure ..... G...
1,https://www.youtube.com/watch?v=tN2j075NLQc&li...,27/07/2020 16:36:05,Kevin Gates - Grandmotha Grave [Official Music...,"8,398,588 views",27 May 2020,NaN,NaN,Monquavis Edwards - Gates hasnt dropped a wack...
2,https://www.youtube.com/watch?v=MgODUPrkVno,27/07/2020 16:36:09,GATE 2020 Computer Science Paper Detailed Solu...,"16,927 views",Streamed live on 8 Feb 2020,218,17,sum 123 - Sir profit % ke liye upon me 3000 ky...
3,https://www.youtube.com/watch?v=MVgCUVf1m90,27/07/2020 16:36:13,GATE 2020 AIR 43 Rahul Sharma,"9,986 views",9 Jun 2020,584,4,Akash Gautam - Who is your real Hero??\nOther ...
4,https://www.youtube.com/watch?v=ybsh1C8QpY0,27/07/2020 16:36:17,How To attempt GATE -Exam-2020 in best way,"306,322 views",31 Jan 2019,8.2K,202,- By Mistake total no of 2 marks questions is...
...,...,...,...,...,...,...,...,...
496,https://www.youtube.com/watch?v=cyRYtFA7PTc,27/07/2020 17:09:07,GATE EE 2020 | How to stay Motivated during Pr...,"1,508 views",Streamed live on 15 Jan 2020,60,2,- Vision 2021: A Course for ESE & GATE Aspira...
497,https://www.youtube.com/watch?v=ZGQSrUzTr5I,27/07/2020 17:09:11,Life after securing AIR 1 in GATE / Why to tar...,"117,631 views",17 Sep 2018,3.9K,256,shivam kumar - Jiya ho bihar ke lala || SHIVEN...
498,https://www.youtube.com/watch?v=mzr3ASusaAs,27/07/2020 17:09:15,GATE 2020 Paper Analysis by GATEFORUM,"1,146 views",31 Jan 2020,9,3,NaN
499,https://youtu.be/XhZ1w3saRiI,27/07/2020 17:09:19,Raipur के Kalpit ने किया कमाल | GATE 2020 में ...,"3,041 views",14 Mar 2020,97,1,NaN


In [111]:
df.shape

(501, 8)

splitting the number of views from the string 'views' and then converting it to an integer

In [112]:
df[['views_num', 'view_str']] = df['Views'].str.split(' ', expand=True)
df['views_num'] = df['views_num'].str.replace(',', '')
df['views_num'] = pd.to_numeric(df['views_num'])

Replaced the str in the Like and dislike columns with NaN values 

In [113]:
df.loc[254, 'Dislikes'] = np.nan
df.loc[254, 'Likes'] = np.nan

Standardazation of the Likes column to remove off the string characters from the numeric characters

In [114]:
def likes_converter(value):
    if pd.isna(value):
        return np.nan
    elif isinstance(value, str) and 'K' in value:
        try:
            return int(float(value.replace('K', '')) * 1000)
        except ValueError:
            return np.nan
    else:
        try:
            return int(value)
        except ValueError:
            return np.nan
    
df = df[~df['Likes'].str.contains('LIE', na=False)]
df['Likes'] = df['Likes'].apply(likes_converter)

Used regex to parse out the date uploaded from the column of upload date which had other characters

In [115]:
date_pattern = r'(\b\d{1,2}\s\w+\s\w{4}\b)'
df['date_uploaded'] = df['upload_date'].str.extract(date_pattern)
df['date_uploaded'] = pd.to_datetime(df['date_uploaded'])

Standardazation of the Dislikes column to remove off the string characters from the numeric characters

In [116]:
def dislike_converter(value):
    if pd.isna(value):
        return np.nan
    elif isinstance(value, str) and 'K' in value:
        try:
            return int(float(value.replace('K', '')) * 1000)
        except ValueError:
            return np.nan
    else:
        try:
            return int(value)
        except ValueError:
            return np.nan
        

df['Dislikes'] = df['Dislikes'].apply(dislike_converter)

In [117]:
df.isnull().sum()

url               0
Timestamp         0
Title             0
Views             0
upload_date       0
Likes            20
Dislikes         20
Comments         64
views_num         0
view_str          0
date_uploaded     0
dtype: int64

In [124]:
def missing_data_percentage(df):
    total_data = df.size
    missing_data = df.isnull().sum().sum()
    percentage = (missing_data / total_data) * 100
    return percentage

print(missing_data_percentage(df))

1.887134821266558


In [126]:
df.dropna(inplace=True)

In [129]:
df.duplicated().sum()

np.int64(0)

In [130]:
df.dtypes

url                      object
Timestamp                object
Title                    object
Views                    object
upload_date              object
Likes                   float64
Dislikes                float64
Comments                 object
views_num                 int64
view_str                 object
date_uploaded    datetime64[ns]
dtype: object

dropped off all the unnecessary columns

In [133]:
keep_col = ['url', 'Title', 'date_uploaded', 'views_num', 'view_str', 'Likes', 'Dislikes', 'Comments']
youtube_clean_data = df[keep_col]

In [134]:
youtube_clean_data.head()

,url,Title,date_uploaded,views_num,view_str,Likes,Dislikes,Comments
0,https://www.youtube.com/watch?v=8g6GpkOvxZ8,"GATE 2020 Topper | Hitesh Poply (CS, AIR-1) | ...",2020-03-14,46489,views,1700.0,32.0,Coding Intelligence - I am pretty sure ..... G...
2,https://www.youtube.com/watch?v=MgODUPrkVno,GATE 2020 Computer Science Paper Detailed Solu...,2020-02-08,16927,views,218.0,17.0,sum 123 - Sir profit % ke liye upon me 3000 ky...
3,https://www.youtube.com/watch?v=MVgCUVf1m90,GATE 2020 AIR 43 Rahul Sharma,2020-06-09,9986,views,584.0,4.0,Akash Gautam - Who is your real Hero??\nOther ...
4,https://www.youtube.com/watch?v=ybsh1C8QpY0,How To attempt GATE -Exam-2020 in best way,2019-01-31,306322,views,8200.0,202.0,- By Mistake total no of 2 marks questions is...
5,https://www.youtube.com/watch?v=fECinEECcqo,GATE 2020 Score vs Opportunities | Suresh VSR,2020-03-14,11682,views,435.0,19.0,VSR Suresh - Thanks students. For the clarific...
